In [1]:
import csv
import networkx as nx
import numpy as np
from random import randint
from random import random
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, accuracy_score
from random import choice
from gensim.models import Word2Vec
import keras
import pandas as pd
from unidecode import unidecode

from sklearn.linear_model import LogisticRegression


from scipy.sparse import identity, diags

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

2023-03-13 21:13:22.629146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
G = nx.read_edgelist('../input_data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
edges = list(G.edges())

print('Number of nodes of total set:', n)
print('Number of edges of total set:', m)

node_to_idx = dict()
for i, node in enumerate(nodes):
    node_to_idx[node] = i

val_edges = list()
G_train = G

for edge in edges:
    if random() < 0.1:
        val_edges.append(edge)

# We remove the val edges from the graph G
for edge in val_edges:
    G_train.remove_edge(edge[0], edge[1])

n = G_train.number_of_nodes()
m = G_train.number_of_edges()
train_edges = list(G_train.edges())
    
print('Number of nodes of training set:', n)
print('Number of edges of training set:', m)

y_val = [1]*len(val_edges)

n_val_edges = len(val_edges)

# Create random pairs of nodes (testing negative edges)
for i in range(n_val_edges):
    n1 = nodes[randint(0, n-1)]
    n2 = nodes[randint(0, n-1)]
    (n1, n2) = (min(n1, n2), max(n1, n2))
    val_edges.append((n1, n2))
    
# Remove from val_edges edges that exist in both train and val
# for edge in list(set(val_edges) & set(train_edges)):
#     val_edges.remove(edge)
    
# n_val_edges = len(val_edges) # - len(y_val) #because we removed from val_edges edges that exist in both
y_val.extend([0]*(n_val_edges))

Number of nodes of total set: 138499
Number of edges of total set: 1091955
Number of nodes of training set: 138499
Number of edges of training set: 982718


In [3]:
def random_walk(G, node, walk_length):
    walk = [node]
  
    for i in range(walk_length-1):
        neibor_nodes = list(G.neighbors(walk[-1]))
        if len(neibor_nodes) > 0:
            next_node = choice(neibor_nodes)
            walk.append(next_node)
    walk = [str(node) for node in walk] # in case the nodes are in string format, we don't need to cast into string, but if the nodes are in numeric or integer, we need this line to cast into string
    return walk

def generate_walks(G, num_walks, walk_length):
  # Runs "num_walks" random walks from each node, and returns a list of all random walk
    walks = list()  
    for i in range(num_walks):
        for node in G.nodes():
            walk = random_walk(G, node, walk_length)
            walks.append(walk)
        #print('walks : ', walks)
    return walks

In [4]:
walks = generate_walks(G=G_train, num_walks=10, walk_length=15)

In [5]:
wv_model = Word2Vec(vector_size=128, window=5, min_count=0, sg=1, workers=8)
wv_model.build_vocab(walks)
wv_model.train(walks, total_examples=wv_model.corpus_count, epochs=5) 

(102878850, 102878850)

In [6]:
# features initializaed with word2vec embedding of the random walks
features_np = []
for node in G_train.nodes():
    features_np.append(wv_model.wv[str(node)])

features_np = np.array(features_np)

In [18]:
# Create class labels
y = np.zeros(4*G_train.number_of_edges())
y[:2*G_train.number_of_edges()] = 1 # Concatenated ones for edges indices and later in the model we add zeros for random indices.


In [44]:
features_np[train_edges[0][0]]
train_edges[300][1]

103345

In [ ]:
# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression()
clf.fit(X_train, y)
y_pred = clf.predict_proba(X_test)
y_pred = y_pred[:,1]

In [32]:
for i in range (10):
    r = randint(0, len(val_edges))
    print(r, val_edges[r], y_val[r])

    
r = len(val_edges)
print('\n',r, val_edges[r-1], y_val[r-1])
# i, element in enumerate(val_edges):
#     print(i, element)
#     break

78297 (43396, 48847) 1
10622 (86116, 110265) 1
159625 (8696, 132796) 0
25512 (33493, 72343) 1
19707 (53809, 65018) 1
144285 (65809, 66383) 0
44028 (21847, 26508) 1
40574 (1502, 83203) 1
147814 (56915, 58622) 0
37875 (11853, 10524) 1

 218474 (9951, 115632) 0


In [33]:
features_np.shape

(138499, 128)